<a href="https://colab.research.google.com/github/dantae74/tensorflow/blob/main/01-auto-graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### @tf.function
* 자동으로 그래프를 생성(Auto Graph)
* 그래프로 변환하여 사용 -> GPU 연산 가능
* 파이썬으로 구성된 함수를 텐서플로우의 그래프 형태로 다루고 싶을 때 사용가능
* 원본함수가 필요하다면 (tf.fucntion).python_function()

In [6]:
@tf.function
def my_function(x):
    return x**2 - 10*x + 3

print(my_function(2))
print(my_function(tf.constant(2)))

tf.Tensor(-13, shape=(), dtype=int32)
tf.Tensor(-13, shape=(), dtype=int32)


In [7]:
def my_function2(x):
    return x**2 - 10*x + 3

print(my_function2(2))
print(my_function2(tf.constant(2)))

-13
tf.Tensor(-13, shape=(), dtype=int32)


In [8]:
tf_my_func = tf.function(my_function2)

print(tf_my_func)
print(tf_my_func(2))

tf.Tensor(-13, shape=(), dtype=int32)


In [9]:
tf_my_func.python_function(2)

-13

In [10]:
def function_to_get_faster(x,y,b):
    x = tf.matmul(x, y)
    x = x + b
    return x

a_function_that_uses_a_graph = tf.function(function_to_get_faster)

x1 = tf.constant([[1.0, 2.0]])
y1 = tf.constant([[2.0],[3.0]])
b1 = tf.constant(4.0)

print(a_function_that_uses_a_graph(x1, y1, b1))
print(a_function_that_uses_a_graph(x1, y1, b1).numpy())

tf.Tensor([[12.]], shape=(1, 1), dtype=float32)
[[12.]]


In [11]:
def inner_function(x,y,b):
    x = tf.matmul(x, y)
    x = x + b
    return x

@tf.function
def outer_function(x):
    y = tf.constant([[2.0],[3.0]])
    b = tf.constant(4.0)
    return inner_function(x,y,b)

print(outer_function(tf.constant([[1.0, 2.0]])))
print(outer_function(tf.constant([[1.0, 2.0]])).numpy())

tf.Tensor([[12.]], shape=(1, 1), dtype=float32)
[[12.]]


In [12]:
print(tf.autograph.to_code(my_function.python_function))
print(tf.autograph.to_code(tf_my_func.python_function))
print(tf.autograph.to_code(outer_function.python_function))

def tf__my_function(x):
    with ag__.FunctionScope('my_function', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()
        try:
            do_return = True
            retval_ = (((ag__.ld(x) ** 2) - (10 * ag__.ld(x))) + 3)
        except:
            do_return = False
            raise
        return fscope.ret(retval_, do_return)

def tf__my_function2(x):
    with ag__.FunctionScope('my_function2', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()
        try:
            do_return = True
            retval_ = (((ag__.ld(x) ** 2) - (10 * ag__.ld(x))) + 3)
        except:
            do_return = False
            raise
        return fscope.ret(retval_, do_return)

def 

In [13]:
import timeit

class SequentialModel(tf.keras.Model):
    def __init__(self, **kwargs):
        super(SequentialModel, self).__init__(**kwargs)
        self.flatten = tf.keras.layers.Flatten(input_shape=(28,28))
        self.dense_1 = tf.keras.layers.Dense(128, activation='relu')
        self.dropout = tf.keras.layers.Dropout(0.2)
        self.dense_2 = tf.keras.layers.Dense(10)
        
    def call(self, x):
        x = self.flatten(x)
        x = self.dense_1(x)
        x = self.dropout(x)
        x = self.dense_2(x)
        return x

input_data = tf.random.uniform([60, 28, 28])

eager_model = SequentialModel()
graph_model = tf.function(eager_model)

In [14]:
print('Eagar time:', timeit.timeit(lambda: eager_model(input_data), number=10000))
print('graph time:', timeit.timeit(lambda: graph_model(input_data), number=10000))

Eagar time: 10.579764436000005
graph time: 6.858694731000014
